# Reach-Scale Modeling Analysis

This Notebook covers preliminary analysis for the design of a simple reach-scale model in TempEst-NEWT/NEXT.

The reach model aims to be as simple as possible while achieving two goals with reasonable accuracy:

- General goal: allow efficient estimation of reach temperature profiles, particularly downstream of observation points or disturbances such as reservoirs.  (Thus, it must be possible to provide arbitrary upstream data, not just a NEWT model.)
- Specific goal: account for key in-channel fluxes such as solar radiation (riparian shading), evaporation, and general atmospheric heat exchange, which cannot be accounted for by a lumped watershed model.  This allows model users to analyze key reach-scale phenomena.

It is anticipated that this can be achieved with a simple equilibrium model, where (for water temperature $T$ and air temperature $T_a$ in C, time $t$, and fitting coefficients $A,~B$):

$$
    \frac{dT}{dt} = A(T_a-T) + B
$$

The fitting coefficient $A$ lumps together a variety of influences on sensible heat flux, such as channel depth.  The fitting coefficient $B$ parametrizes a number of effects where their variation with stream temperature probably cannot be estimated for an ungaged watershed, and which are therefore assumed to be constant (with respect to temperature, not time).  These include, chiefly, solar radiation, evaporation, and lateral/groundwater inflows.

For model training purposes, we do not have densely-spaced observations, and it is necessary to apply uniform coefficients.  Over long distances, applying the above equation linearly would result in substantial error, so we use an exponential solution:

$$
    T - T_a = C\exp(-At) + Bt \implies T = C\exp(-At) + Bt + T_a \implies \Delta T = T_0-T = (C + T_a) - (C\exp(-At) + Bt + T_a) = C(1-\exp(-At)) - Bt
$$

Such that:

- If $B$ is zero, $T$ exponentially decays towards $T_a$
- C = $T-T_a$ at the initial step
- A and B are fitted

A further necessity is to estimate the travel time over a reach, $t$.  With uniform flow, we can begin from Manning's equation, $v=\frac1n R^{2/3}S^{1/2}$ (for velocity $v$, roughness $n$, hydraulic radius $R$, and slope $S$).  Assuming - somewhat dubiously, but of necessity without coupling a hydrologic model - that $R$ has a power-law relationship to flow $Q$, which in turn has a power law relationship to the product of drainage area $D$ and precipitation $P$, we have $v=\frac1{\alpha}\frac1n(DP)^{\beta}S^{1/2}$ for fitting coefficients $\alpha,~\beta$.  Thus, for reach length $L$ and fitting coefficient 

$$
    t = \frac{\alpha Ln}{(DP)^{\beta}\sqrt{S}}
$$

Now, for the CONUS, we can get $n$, $L$, $S$ from NHDPlus Value Added Attributes, and only need to estimate $\alpha,~\beta$.

Note that for fitting a single river, those are all constant, so we (introducing a few more fitting coefficients, because we don't have enough to keep track of) simply have $t=\frac{k}{(DP)^q}$, and therefore need to fit $A,~B,~k,~q$ (we have an analytic solution for $C$).  Fitting four coefficients is pretty easy with a reasonably long timeseries, and since we're using a simple analytic solution, we can very quickly calculate tons of permutations and identify a rock-solid optimum.  If our assumptions are sufficiently decent, of course.

Now, how do we get the training data?  We want to use the reach model to capture details that the lumped model can't, so we're going to train off of USGS observations, which is why we can't get high-resolution training data.  But we do want our training data to represent a range of reach lengths, so we don't miss long-range or short-range effects.  So, over the 900ish development gages, we:

1. Set a random reach length as an integer kilometer between 1-100 km, uniformly distributed on a log scale (i.e., 1, 10, 100 not 1, 50, 100).  Longer than 100 km is past what would likely be of interest, and far too long to be anything like uniform; shorter than 1 km is getting into the range where detailed hydrology and hydraulics inaccessible to NEXT would be important.  Note that it'll be necessary to check the actual mainstem length, since some sites may have less than 100 km upstream.  I am going to arbitrarily set the maximum length to be 10% of the mainstem, so that most flow comes from the upper watershed; otherwise our Q~AP assumption would be even more problematic.
2. Compute flow time in the reach.
3. Compute the "no-reach" temperature as the area-weighted mean of all contributing watershed temperatures, with the timestep offset if flow time is more than half a day.
4. Compute the "missing" temperature as the difference between observed and no-reach temperature.
5. Using the first 70% of the available timeseries, calibrate the reach model coefficients to the "missing" temperature.
6. Test that solution on the final 30%.
7. Develop coefficient estimation.

Using what we've derived above, the final solution for reachwise *change in temperature* is:

$$
    \Delta T =  (T_0-T_a)\left(1-\exp\left(-\frac{Ak}{(DP)^q}\right)\right) - \frac{Bk}{(DP)^q}
$$

Note that $k$, $q$ are constants, while $A$ varies with discharge and $B$ with... everything.

Also note that the precipitation-area assumption *does not* work during major snowfall and snowmelt.  I'm not sure how to handle that.

Before beginning the full process above, I will use a single test reach to evaluate model design and make sure it works, generally.